In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *
import gc
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
import zipfile
from sklearn import preprocessing 

In [31]:
ss = pd.read_csv('C:\\Users\\Himanshu\\Downloads\\sample_submission_lfbv3c3.csv')
zf = zipfile.ZipFile('C:\\Users\\Himanshu\\Downloads\\Test_ND2Q3bm.zip') 
test = pd.read_csv(zf.open('test.csv'))
zf1 = zipfile.ZipFile('C:\\Users\\Himanshu\\Downloads\\Train_hMYJ020.zip')
train = pd.read_csv(zf1.open('train.csv'))
description = pd.read_csv(zf1.open('train_data_dict.csv'))

In [32]:
def display_data_information(data, data_types, dataframe_name):
    print(" Information of ",dataframe_name,": Rows = ",data.shape[0],"| Columns = ",data.shape[1],"\n")
    data.info()
    print("\n")
    for VARIABLE in data_types :
        data_type = data.select_dtypes(include=[ VARIABLE ]).dtypes
        if len(data_type) > 0 :
            print(str(len(data_type))+" "+VARIABLE+" Features\n"+str(data_type)+"\n"  )        


data_types  = ["float32","float64","int32","int64","object","category","datetime64[ns]"]
display_data_information(train, data_types, "train")

 Information of  train : Rows =  318438 | Columns =  18 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   Hospital_type_code                 318438 non-null  object 
 3   City_Code_Hospital                 318438 non-null  int64  
 4   Hospital_region_code               318438 non-null  object 
 5   Available Extra Rooms in Hospital  318438 non-null  int64  
 6   Department                         318438 non-null  object 
 7   Ward_Type                          318438 non-null  object 
 8   Ward_Facility_Code                 318438 non-null  object 
 9   Bed Grade                          318325 non-null  float64
 10  patientid                          318438 non-

In [33]:
def display_head_tail(data, head_rows, tail_rows):
    display("Data Head & Tail :")
    display(data.head(head_rows).append(data.tail(tail_rows)))

display_head_tail(train, head_rows=4, tail_rows=4)

'Data Head & Tail :'

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
318434,318435,24,a,1,X,2,anesthesia,Q,E,4.0,325,8.0,Urgent,Moderate,4,81-90,6699.0,31-40
318435,318436,7,a,4,X,3,gynecology,R,F,4.0,125235,10.0,Emergency,Minor,3,71-80,4235.0,11-20
318436,318437,11,b,2,Y,3,anesthesia,Q,D,3.0,91081,8.0,Trauma,Minor,5,11-20,3761.0,11-20
318437,318438,19,a,7,Y,5,gynecology,Q,C,2.0,21641,8.0,Emergency,Minor,2,11-20,4752.0,0-10


In [34]:
def display_data_description(data, numeric_data_types, categorical_data_types):
    
    print("Data Description :")
    display(data.describe( include = numeric_data_types))
    print("")
    display(data.describe( include = categorical_data_types))

display_data_description(train, data_types[0:4], data_types[4:7])

Data Description :


,case_id,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,patientid,City_Code_Patient,Visitors with Patient,Admission_Deposit
count,318438.000000,318438.000000,318438.000000,318438.000000,318325.000000,318438.000000,313906.000000,318438.000000,318438.000000
mean,159219.500000,18.318841,4.771717,3.197627,2.625807,65747.579472,7.251859,3.284099,4880.749392
std,91925.276847,8.633755,3.102535,1.168171,0.873146,37979.936440,4.745266,1.764061,1086.776254
min,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1800.000000
25%,79610.250000,11.000000,2.000000,2.000000,2.000000,32847.000000,4.000000,2.000000,4186.000000
50%,159219.500000,19.000000,5.000000,3.000000,3.000000,65724.500000,8.000000,3.000000,4741.000000
75%,238828.750000,26.000000,7.000000,4.000000,3.000000,98470.000000,8.000000,4.000000,5409.000000
max,318438.000000,32.000000,13.000000,24.000000,4.000000,131624.000000,38.000000,32.000000,11008.000000


,Hospital_type_code,Hospital_region_code,Department,Ward_Type,Ward_Facility_Code,Type of Admission,Severity of Illness,Age,Stay
count,318438,318438,318438,318438,318438,318438,318438,318438,318438
unique,7,3,5,6,6,3,3,10,11
top,a,X,gynecology,R,F,Trauma,Moderate,41-50,21-30
freq,143425,133336,249486,127947,112753,152261,175843,63749,87491


In [35]:
display_data_information(test, data_types, "test")

 Information of  test : Rows =  137057 | Columns =  17 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137057 entries, 0 to 137056
Data columns (total 17 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            137057 non-null  int64  
 1   Hospital_code                      137057 non-null  int64  
 2   Hospital_type_code                 137057 non-null  object 
 3   City_Code_Hospital                 137057 non-null  int64  
 4   Hospital_region_code               137057 non-null  object 
 5   Available Extra Rooms in Hospital  137057 non-null  int64  
 6   Department                         137057 non-null  object 
 7   Ward_Type                          137057 non-null  object 
 8   Ward_Facility_Code                 137057 non-null  object 
 9   Bed Grade                          137022 non-null  float64
 10  patientid                          137057 non-n

In [36]:
display_head_tail(test, head_rows=4, tail_rows=4 )

'Data Head & Tail :'

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,318439,21,c,3,Z,3,gynecology,S,A,2.0,17006,2.0,Emergency,Moderate,2,71-80,3095.0
1,318440,29,a,4,X,2,gynecology,S,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4018.0
2,318441,26,b,2,Y,3,gynecology,Q,D,4.0,17006,2.0,Emergency,Moderate,3,71-80,4492.0
3,318442,6,a,6,X,3,gynecology,Q,F,2.0,17006,2.0,Trauma,Moderate,3,71-80,4173.0
137053,455492,25,e,1,X,2,radiotherapy,R,E,4.0,30985,7.0,Emergency,Moderate,2,0-10,3510.0
137054,455493,30,c,3,Z,2,anesthesia,R,A,4.0,81811,12.0,Urgent,Minor,2,0-10,7190.0
137055,455494,5,a,1,X,2,anesthesia,R,E,4.0,57021,10.0,Trauma,Minor,2,41-50,5435.0
137056,455495,6,a,6,X,3,gynecology,Q,F,4.0,126729,3.0,Trauma,Extreme,5,51-60,4702.0


In [37]:
display_data_description(test, data_types[0:4], data_types[4:7])

Data Description :


,case_id,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,patientid,City_Code_Patient,Visitors with Patient,Admission_Deposit
count,137057.000000,137057.000000,137057.000000,137057.000000,137022.000000,137057.000000,134900.000000,137057.000000,137057.000000
mean,386967.000000,18.343747,4.758692,3.192686,2.634489,65877.903515,7.243996,3.284531,4869.731097
std,39565.092259,8.634694,3.102245,1.164250,0.869295,37942.997623,4.790625,1.777270,1080.766723
min,318439.000000,1.000000,1.000000,0.000000,1.000000,3.000000,1.000000,0.000000,1800.000000
25%,352703.000000,11.000000,2.000000,2.000000,2.000000,32945.000000,4.000000,2.000000,4178.000000
50%,386967.000000,19.000000,5.000000,3.000000,3.000000,65786.000000,8.000000,3.000000,4731.000000
75%,421231.000000,26.000000,7.000000,4.000000,3.000000,98851.000000,8.000000,4.000000,5398.000000
max,455495.000000,32.000000,13.000000,21.000000,4.000000,131618.000000,38.000000,32.000000,11920.000000


,Hospital_type_code,Hospital_region_code,Department,Ward_Type,Ward_Facility_Code,Type of Admission,Severity of Illness,Age
count,137057,137057,137057,137057,137057,137057,137057,137057
unique,7,3,5,6,6,3,3,10
top,a,X,gynecology,R,F,Trauma,Moderate,41-50
freq,61305,57513,107202,54992,48717,65411,75722,27746


In [38]:
def remove_duplicate(data):
    
    print("BEFORE REMOVING DUPLICATES - No. of Rows = ",data.shape[0])
    data.drop_duplicates(keep="first", inplace=True) 
    print("AFTER REMOVING DUPLICATES  - No. of Rows = ",data.shape[0])
    
    return data

train = remove_duplicate(train)

BEFORE REMOVING DUPLICATES - No. of Rows =  318438
AFTER REMOVING DUPLICATES  - No. of Rows =  318438


In [39]:
train.isnull().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             113
patientid                               0
City_Code_Patient                    4532
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

In [40]:
test.isnull().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                              35
patientid                               0
City_Code_Patient                    2157
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
dtype: int64

In [41]:
def display_unique(data):
    for column in data.columns :
        
        print("No of Unique Values in "+column+" Column are : "+str(data[column].nunique()))
        print("Actual Unique Values in "+column+" Column are : "+str(data[column].sort_values(ascending=True,na_position='last').unique() ))
        print("NULL Values :")
        print(data[ column ].isnull().sum())
        print("Value Counts :")
        print(data[column].value_counts())
        print("")

display_unique(train)

No of Unique Values in case_id Column are : 318438
Actual Unique Values in case_id Column are : [     1      2      3 ... 318436 318437 318438]
NULL Values :
0
Value Counts :
2047      1
107105    1
31322     1
29275     1
19036     1
         ..
21920     1
23969     1
17826     1
19875     1
2049      1
Name: case_id, Length: 318438, dtype: int64

No of Unique Values in Hospital_code Column are : 32
Actual Unique Values in Hospital_code Column are : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
NULL Values :
0
Value Counts :
26    33076
23    26566
19    21219
6     20425
11    17328
14    17328
28    17137
27    14244
9     11510
29    11311
12    11297
32    10703
25     9834
10     9435
15     9257
21     8150
24     7992
3      7116
17     5501
5      5261
1      5249
13     5236
2      5102
30     5002
22     4277
31     3967
16     3671
8      3663
18     3630
20     1405
7      1306
4      1240
Name: Hospital_code, dtype: in

In [42]:
train['City_Code_Patient']=train['City_Code_Patient'].replace('NaN',40)
train['Bed Grade']=train['Bed Grade'].replace('NaN',5)
test['City_Code_Patient']=test['City_Code_Patient'].replace('NaN',40)
test['Bed Grade']=test['Bed Grade'].replace('NaN',5)

In [43]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in ['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission','Severity of Illness','Age']:
    train[col]= train[col].astype('str')
    train[col]= le.fit_transform(train[col])

In [44]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in ['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission','Severity of Illness','Age']:
    test[col]= test[col].astype('str')
    test[col]= le.fit_transform(test[col])

In [45]:
display_data_information(train, data_types, "train")

 Information of  train : Rows =  318438 | Columns =  18 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318438 entries, 0 to 318437
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   Hospital_type_code                 318438 non-null  int32  
 3   City_Code_Hospital                 318438 non-null  int64  
 4   Hospital_region_code               318438 non-null  int32  
 5   Available Extra Rooms in Hospital  318438 non-null  int64  
 6   Department                         318438 non-null  int32  
 7   Ward_Type                          318438 non-null  int32  
 8   Ward_Facility_Code                 318438 non-null  int32  
 9   Bed Grade                          318325 non-null  float64
 10  patientid                          318438 non-

In [46]:
display_data_information(test, data_types, "test")

 Information of  test : Rows =  137057 | Columns =  17 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137057 entries, 0 to 137056
Data columns (total 17 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            137057 non-null  int64  
 1   Hospital_code                      137057 non-null  int64  
 2   Hospital_type_code                 137057 non-null  int32  
 3   City_Code_Hospital                 137057 non-null  int64  
 4   Hospital_region_code               137057 non-null  int32  
 5   Available Extra Rooms in Hospital  137057 non-null  int64  
 6   Department                         137057 non-null  int32  
 7   Ward_Type                          137057 non-null  int32  
 8   Ward_Facility_Code                 137057 non-null  int32  
 9   Bed Grade                          137022 non-null  float64
 10  patientid                          137057 non-n

In [47]:
np.intersect1d(train['case_id'], test['case_id']).shape

(0,)

In [48]:
var = train[~(train.Admission_Deposit>8000)]
var1 = var[~(var.Admission_Deposit<2500)]
var2 = var1[~(var1['Available Extra Rooms in Hospital']>20)]
var3 = var2[~(var2['Visitors with Patient']>22)]

In [49]:
var3

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,2,3,2,3,3,2,5,2.0,31397,7.0,0,0,2,5,4911.0,0-10
1,2,2,2,5,2,2,3,3,5,2.0,31397,7.0,1,0,2,5,5954.0,41-50
2,3,10,4,1,0,2,1,3,4,2.0,31397,7.0,1,0,2,5,4745.0,31-40
3,4,26,1,2,1,2,3,2,3,2.0,31397,7.0,1,0,2,5,7272.0,41-50
4,5,26,1,2,1,2,3,3,3,2.0,31397,7.0,1,0,2,5,5558.0,41-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,318434,6,0,6,0,3,3,1,5,4.0,86499,23.0,0,2,3,4,4144.0,11-20
318434,318435,24,0,1,0,2,1,1,4,4.0,325,8.0,2,2,4,8,6699.0,31-40
318435,318436,7,0,4,0,3,2,2,5,4.0,125235,10.0,0,1,3,7,4235.0,11-20
318436,318437,11,1,2,1,3,1,1,3,3.0,91081,8.0,1,1,5,1,3761.0,11-20


In [50]:
X = var3.drop(['patientid','case_id','Stay'], axis = 'columns')
y = pd.DataFrame(var3['Stay'])

In [51]:
scaler = preprocessing.StandardScaler() 
standard_df = scaler.fit_transform(X) 
standard_df = pd.DataFrame(standard_df) 

In [52]:
cat_3 = CatBoostClassifier()

In [53]:
cat_3.fit(standard_df,y)

Learning rate set to 0.106052
0:	learn: 2.2156550	total: 2.76s	remaining: 46m 1s
1:	learn: 2.0971599	total: 4.07s	remaining: 33m 51s
2:	learn: 2.0097792	total: 5.33s	remaining: 29m 31s
3:	learn: 1.9418583	total: 6.51s	remaining: 27m
4:	learn: 1.8868305	total: 7.73s	remaining: 25m 37s
5:	learn: 1.8433629	total: 8.94s	remaining: 24m 41s
6:	learn: 1.8047746	total: 10.2s	remaining: 24m
7:	learn: 1.7735328	total: 11.3s	remaining: 23m 25s
8:	learn: 1.7463784	total: 12.5s	remaining: 23m
9:	learn: 1.7241037	total: 13.7s	remaining: 22m 38s
10:	learn: 1.7052864	total: 14.9s	remaining: 22m 20s
11:	learn: 1.6889106	total: 16.1s	remaining: 22m 4s
12:	learn: 1.6728571	total: 17.3s	remaining: 21m 54s
13:	learn: 1.6590306	total: 18.6s	remaining: 21m 47s
14:	learn: 1.6485251	total: 19.8s	remaining: 21m 39s
15:	learn: 1.6375355	total: 21s	remaining: 21m 31s
16:	learn: 1.6289205	total: 22.3s	remaining: 21m 29s
17:	learn: 1.6216195	total: 23.6s	remaining: 21m 26s
18:	learn: 1.6151732	total: 25s	remaining:

In [54]:
cat_3.score(standard_df,y)

0.45537085423913565

In [55]:
b = pd.DataFrame(test.case_id)

In [56]:
test.drop(['case_id','patientid'], axis ='columns', inplace = True)
scaler1 = preprocessing.StandardScaler() 
standard_df1 = scaler1.fit_transform(test) 
standard_df1 = pd.DataFrame(standard_df1) 

In [57]:
a = pd.DataFrame(cat_3.predict(standard_df1))
sub = pd.concat([b, a], axis = 1)
sub.rename(columns = {0 : 'Stay'}, inplace = True)
sub.head(10)

,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60
5,318444,21-30
6,318445,21-30
7,318446,11-20
8,318447,21-30
9,318448,21-30


In [58]:
sub.to_csv('C:\\Users\\Himanshu\\Downloads\\sub.csv',index = None, header = True, encoding = 'UTF-8')